In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import FastICA  # Change to FastICA for ICA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.compose import ColumnTransformer

In [2]:
# Load data
df = pd.read_csv("diabetes_prediction_dataset.csv")

# Handle duplicates
duplicate_rows_data = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_data.shape)
df = df.drop_duplicates()

# Remove Unnecessary value [0.00195%]
df = df[df['gender'] != 'Other']


number of duplicate rows:  (3854, 9)


In [3]:
# Recategorize smoking history
def recategorize_smoking(smoking_status):
    if smoking_status in ['never', 'No Info']:
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'

df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)


In [4]:
# One-hot encoding
def perform_one_hot_encoding(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df.drop(column_name, axis=1), dummies], axis=1)
    return df

data = df.copy()
data = perform_one_hot_encoding(data, 'gender')
data = perform_one_hot_encoding(data, 'smoking_history')


In [5]:
# Resampling
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

# Preprocessing with ICA
preprocessor_ica = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level', 'hypertension', 'heart_disease']),
        ('cat', OneHotEncoder(), ['gender', 'smoking_history'])
    ])

ica = FastICA(n_components=5)  # Adjust the number of components based on your requirements


In [6]:
# SelectKBest with Mutual Information for feature selection
feature_selection = SelectKBest(mutual_info_classif, k=5)  # Adjust k based on your requirements

# Define k-NN classifier
knn_classifier = KNeighborsClassifier()


In [7]:
# Create a pipeline that preprocesses the data, performs feature selection, resamples data, and then trains a classifier
clf_ica = imbPipeline(steps=[
    ('preprocessor_ica', preprocessor_ica),
    ('ica', ica),
    ('feature_selection', feature_selection),
    ('over', over),
    ('under', under),
    ('classifier', knn_classifier)
])

param_grid_ica = {
    'classifier__n_neighbors': [25],
    'classifier__algorithm': ['auto'],
    'classifier__leaf_size': [5],
    'classifier__weights': ['uniform'],
    'classifier__p': [2]  # L2-norm (Euclidean)
}

# Create Grid Search object for ICA
grid_search_ica = GridSearchCV(clf_ica, param_grid_ica, cv=5)

In [8]:
# Split data into train and test sets
X = df.drop('diabetes', axis=1)
y = df['diabetes']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Train the model with ICA
grid_search_ica.fit(X_train, y_train)

# Print the best parameters with ICA
print("Best Parameters with ICA: ", grid_search_ica.best_params_)

Best Parameters with ICA:  {'classifier__algorithm': 'auto', 'classifier__leaf_size': 5, 'classifier__n_neighbors': 25, 'classifier__p': 2, 'classifier__weights': 'uniform'}


In [10]:
# Predict on the test set using the best model with ICA
y_pred_ica = grid_search_ica.predict(X_test)

# Evaluate the model with ICA
print("Model Accuracy with ICA: ", accuracy_score(y_test, y_pred_ica))
print(classification_report(y_test, y_pred_ica))

Model Accuracy with ICA:  0.9336315406220743
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     17525
           1       0.59      0.82      0.68      1701

    accuracy                           0.93     19226
   macro avg       0.79      0.88      0.82     19226
weighted avg       0.95      0.93      0.94     19226

